# Increasing the Maximum Message Size for gRPC


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)


# Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

**2018-06-13** : At present we find the most stable version of minikube across platforms is 0.25.2 as there are issues with 0.26 and 0.27 on some systems. We also find the default VirtualBox driver can be problematic on some systems to we suggest using the [KVM2 driver](https://github.com/kubernetes/minikube/blob/master/docs/drivers.md#kvm2-driver).

Your start command would then look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

# Setup

In [1]:
!kubectl create namespace seldon

namespace "seldon" created


In [1]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding "kube-system-cluster-admin" created


# Install Helm

In [2]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


## Start seldon-core

Install the custom resource definition

In [3]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Mon Aug 27 20:10:36 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s


NOTES:
NOTES: TODO




In [46]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Mon Aug 27 11:39:49 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/RoleBinding
NAME    AGE
seldon  0s

==> v1/Pod(related)
NAME                                                READY  STATUS             RESTARTS  AGE
seldon-core-ambassador-7fb4575f6b-8rqfv             0/2    ContainerCreating  0         0s
seldon-core-seldon-apiserver-68f5984b49-vxbcd       0/1    ContainerCreating  0         0s
seldon-core-seldon-cluster-manager-9fdd5d6df-9rs46  0/1    Pending            0         0s
seldon-core-seldon-cluster-manager-9fdd5d6df-wkbjv  0/1    Terminating        0         10m
seldon-core-redis-575979b79b-85cwn                  0/1    Pending            0         0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          0s
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldo

Check all services are running before proceeding.

In [23]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward to API Gateway**

REST:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

GRPC:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8005:8080
```

Install gRPC modules for the prediction protos.

In [24]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [130]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import numpy as np

AMBASSADOR_API="localhost:8005"
GATEWAY_REST="localhost:8003"
GATEWAY_GRPC="localhost:8004"

def get_token(oauth_key,oauth_secret):
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+GATEWAY_REST+"/oauth/token",
                auth=HTTPBasicAuth(oauth_key, oauth_secret),
                data=payload)
    print(response.text)
    token =  response.json()["access_token"]
    return token

def rest_request_api_gateway():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+GATEWAY_REST+"/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print(response.text)

def grpc_request_api_gateway(oauth_key,oauth_secret,data_size):
    token = get_token(oauth_key,oauth_secret)
    shape = [1,data_size]
    arr = np.random.rand(data_size)
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = shape,
                values = arr
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(GATEWAY_GRPC)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)
    
def grpc_request_ambassador(deploymentName,data_size):
    shape = [1,data_size]
    arr = np.random.rand(data_size)
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = shape,
                values = arr
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(AMBASSADOR_API)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('seldon',deploymentName)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)



In [26]:
!pygmentize resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
          

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [48]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment "seldon-deployment-example" created


In [ ]:
!kubectl get seldondeployments -n seldon

In [ ]:
!kubectl describe seldondeployments seldon-deployment-example -n seldon

Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

## Get predictions - no grpx max message size

Send a small request which should suceed.

In [98]:
grpc_request_api_gateway('oauth-key','oauth-secret',10)

{"access_token":"d7deafbe-a345-4979-8b67-56e5902f80f4","token_type":"bearer","expires_in":34537,"scope":"read write"}
meta {
  puid: "ldbh86ikegf56hvrke0n5lfsk4"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08242582139811103
  }
}



Send a large request which will be above the default gRPC message size and will fail.

In [99]:
grpc_request_api_gateway('oauth-key','oauth-secret',1000000)

{"access_token":"d7deafbe-a345-4979-8b67-56e5902f80f4","token_type":"bearer","expires_in":34511,"scope":"read write"}


_Rendezvous: <_Rendezvous of RPC that terminated with (StatusCode.CANCELLED, Received RST_STREAM with error code 8)>

In [100]:
!kubectl delete -f resources/model.json

seldondeployment "seldon-deployment-example" deleted


In [103]:
!helm delete seldon-core --purge

release "seldon-core" deleted


# Allowing larger gRPC messages

Recreate seldon-core with extra annotation for the API Gateway

In [4]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true \
    -f grpc_size_values.yaml

NAME:   seldon-core
LAST DEPLOYED: Mon Aug 27 20:10:54 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Role
NAME          AGE
seldon-local  0s
ambassador    0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador        NodePort   10.106.254.3    <none>       8080:31937/TCP                 0s
seldon-core-ambassador-admin  NodePort   10.97.45.246    <none>       8877:31029/TCP                 0s
seldon-core-seldon-apiserver  NodePort   10.104.137.8    <none>       8080:32165/TCP,5000:30238/TCP  0s
seldon-core-redis             ClusterIP  10.106.213.160  <none>       6379/TCP                       0s

==> v1/ClusterRoleBinding
NAME    AGE
seldon  0s

==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  0s

==> v1/RoleBinding
NAME    AGE
seldon  0s

==> v1beta1/RoleBinding
ambassador  0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABL

Wait for seldon core deployment to be running

In [106]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
deployment "seldon-core-ambassador" successfully rolled out


Now we change our SeldonDeployment to include a annotation for max grpx message size.

In [137]:
!pygmentize resources/model_grpc_size.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1",
	    "seldon.io/grpc-max-message-size":"10000000"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_grpc:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
          

In [117]:
!kubectl create -f resources/model_grpc_size.json -n seldon

seldondeployment "seldon-deployment-example" created


**Make sure you rerun the port forwards above as a new seldon core API gateway and Ambassador will have started**

Send a large message. This time it should succeed.

In [129]:
grpc_request_api_gateway('oauth-key','oauth-secret',900000)

{"access_token":"34fa5f9b-9707-4b98-88fb-5798ebcc1fdb","token_type":"bearer","expires_in":30636,"scope":"read write"}
meta {
  puid: "huc201rtjhvff4dcvh3nnchp7u"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.081916313487403
  }
}



Send a request via ambassador. This should also succeed.

In [136]:
grpc_request_ambassador("seldon-deployment-example",900000)

meta {
  puid: "3bddiv4gtr1881gh68dnv4am3l"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08191186455491338
  }
}

